<a href="https://colab.research.google.com/github/kaushkay/midass-iiitd/blob/master/CV_Problem/%20CV_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
# Loading Dataset.

from zipfile import ZipFile
file_name = 'Vision_task_dataset_public-20190406T201525Z-001.zip'

with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print("Done")

Done


In [0]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np

In [0]:
import tensorflow as tf

In [0]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

In [0]:
train_images = pd.read_pickle('Vision_task_dataset_public/train_image.pkl')

In [0]:
train_labels = pd.read_pickle('Vision_task_dataset_public/train_label.pkl')

In [0]:
test_images = pd.read_pickle('Vision_task_dataset_public/test_image.pkl')

In [0]:
x_train = np.array(train_images)

In [0]:
def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot


In [89]:
x_train.shape

(8000, 784)

In [0]:
x_labels = np.array(train_labels)

In [91]:
x_labels.shape

(8000,)

In [0]:
x_test = np.array(test_images)

In [93]:
x_test.shape

(2000, 784)

In [0]:
X_TRAIN, X_VAL, Y_TRAIN, Y_VAL = train_test_split(x_train, x_labels, test_size=0.2)

In [95]:
X_TRAIN.shape

(6400, 784)

In [96]:
X_VAL.shape

(1600, 784)

In [97]:
Y_TRAIN.shape

(6400,)

In [0]:
Y_TRAIN = dense_to_one_hot(Y_TRAIN, 4)

In [99]:
Y_TRAIN.shape

(6400, 4)

In [0]:
Y_VAL = dense_to_one_hot(Y_VAL, 4)

In [101]:
Y_VAL.shape

(1600, 4)

In [0]:
# Hyper-Parameters
learning_rate = 0.01
num_steps = 100

In [0]:
# Network Parameters
n_hidden_1 = 30
n_hidden_2 = 30
#n_hidden_3 = 50
num_input = 784
num_classes = 4

In [0]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [0]:
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    #'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    #'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    #layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [0]:
# Construct model
logits = neural_net(X)
prediction = tf.nn.softmax(logits)

In [0]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [0]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()

In [385]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    costs = []
    for epoch in range(1, num_steps+1):
        cost = 0
  
        _, c = sess.run([train_op, loss_op], feed_dict={X: X_TRAIN, Y:Y_TRAIN})
        
        
        cost = c/6400
        costs.append(cost)
        
        print("Epoch:", (epoch), "cost =", "{:.5f}".format(cost))
        
    print("\nTraining complete!")
    
    
    pred_temp = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    print("Validation Accuracy:", accuracy.eval({X: X_VAL.reshape(-1, num_input), Y: Y_VAL}))
    
    predict = tf.argmax(logits, 1)
    pred = predict.eval({X: x_test.reshape(-1, num_input)})

Epoch: 1 cost = 13.32811
Epoch: 2 cost = 5.88546
Epoch: 3 cost = 3.23994
Epoch: 4 cost = 4.49352
Epoch: 5 cost = 4.87924
Epoch: 6 cost = 3.84457
Epoch: 7 cost = 2.63428
Epoch: 8 cost = 2.34442
Epoch: 9 cost = 2.48041
Epoch: 10 cost = 2.58157
Epoch: 11 cost = 2.79044
Epoch: 12 cost = 2.73811
Epoch: 13 cost = 2.32232
Epoch: 14 cost = 1.88829
Epoch: 15 cost = 1.61913
Epoch: 16 cost = 1.49300
Epoch: 17 cost = 1.48301
Epoch: 18 cost = 1.55788
Epoch: 19 cost = 1.67308
Epoch: 20 cost = 1.75020
Epoch: 21 cost = 1.71401
Epoch: 22 cost = 1.56493
Epoch: 23 cost = 1.39231
Epoch: 24 cost = 1.28908
Epoch: 25 cost = 1.28968
Epoch: 26 cost = 1.37379
Epoch: 27 cost = 1.48955
Epoch: 28 cost = 1.52844
Epoch: 29 cost = 1.43868
Epoch: 30 cost = 1.28197
Epoch: 31 cost = 1.15959
Epoch: 32 cost = 1.11420
Epoch: 33 cost = 1.11750
Epoch: 34 cost = 1.12474
Epoch: 35 cost = 1.11241
Epoch: 36 cost = 1.06779
Epoch: 37 cost = 1.06488
Epoch: 38 cost = 1.11318
Epoch: 39 cost = 1.08491
Epoch: 40 cost = 1.00951
Epoch: 4

In [0]:
costs = np.array(costs)

In [336]:
costs.min()

417.0355859375

In [0]:
 # find predictions on val set

In [143]:
with tf.Session() as sess:
    
    pred_temp = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    print("Validation Accuracy:", accuracy.eval({X: X_VAL.reshape(-1, num_input), Y: Y_VAL}))

FailedPreconditionError: ignored